In [4]:
%matplotlib inline
%load_ext autoreload

import warnings
warnings.filterwarnings("ignore") # disable warnings

from os import getcwd
from os.path import join, abspath, pardir, exists
import numpy as np
import pandas as pd

import pickle, json

import matplotlib.pyplot as plt
import seaborn as sns

# plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# scipy
from scipy.stats import ttest_ind, chi2_contingency, boxcox, skew
from scipy.stats.stats import pearsonr

# sklearn libraries
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import SelectKBest, f_classif, chi2

# IPython
from IPython.core.interactiveshell import InteractiveShell

# statsmodel
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Config settings

In [3]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, "data")
data_file = join(data_dir, "speed_dating.csv")

# For IPython

InteractiveShell.ast_node_interactivity = "all" # To show all output after each cell execution (instead of the last output)

# For pandas

pd.options.display.max_columns = 200 # display upto 200 columns (instead of default 20)
pd.options.display.max_rows = 200 # display upto 200 rows (instead of default 60)

#### Helper functions

In [ ]:
def save_model(model, file_path: str) -> None:
    """
    Save model as a pickle file
    """
    with open(file_path, "wb") as file:
        pickle.dump(model, file)

def load_model(file_path: str):
    """
    Load model from a pickle file
    """
    with open(file_path, "rb") as file:
        return pickle.load(file)

def dataframe_to_csv(df: pd.DataFrame, file_path: str) -> None:
    """
    Save dataframe as .csv file
    """
    df.to_csv(file_path, index=False)

### Pre-processing

Data pre-processing is the first machine learning step in which we transform raw data obtained from various sources into a usable format to implement accurate machine learning models. 

In a real-world data science project, data pre-processing is one of the most important things, and it is one of the common factors of success of a model, i.e., if there is correct data pre-processing and feature engineering, that model is more likely to produce noticeably better results as compared to a model for which data is not well preprocessed.

That's why you may have heard that 80% of a data scientist’s time goes into data pre-processing and 20% of the time for model building. This isn’t false and is actually the case.



In [ ]:
df = pd.read_csv(data_file, encoding= 'ISO-8859-1')
df.head()

##### Basic checks

In [ ]:
len(df)
len(df.columns)
# df.dtypes

##### Missing data

In [ ]:
((df.isna().sum()/len(df)) * 100).to_frame(name='missing %').sort_values(by=['missing %'], ascending=False)

Below mentioned are the articles, blogs, papers and other resources which we took inspiration from or we think are useful for future use case

#### References

- [_Getting Started with Data Preprocessing in Python_](https://www.section.io/engineering-education/data-preprocessing-python/)
- [_Data Preprocessing in Python_](https://medium.datadriveninvestor.com/data-preprocessing-3cd01eefd438)
- [_Data Pre-processing in Python for Beginner_](https://medium.com/data-science-indo/data-preparation-in-python-for-beginner-d3e1e60c03a6)
- [_Data Preprocessing using Python_](https://medium.com/@suneet.bhopal/data-preprocessing-using-python-1bfee9268fb3)
- [_Data preprocessing for Machine Learning in Python_](https://towardsdatascience.com/data-preprocessing-for-machine-learning-in-python-2d465f83f18c)
- [_Data Preprocessing in Machine Learning_](https://www.analytixlabs.co.in/blog/data-preprocessing-in-machine-learning/)